In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import json
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Matus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

**Calculating additional features to use in the analysis.**
- input: formatted results and respondents cognitive tasks data
- output: final results and respondents exports to be used for statistical analysis

# Calculating features

## Calculating new result features

**Load results.**

In [2]:
results = pd.read_csv("../data/results_labeled.csv", index_col=0)

In [3]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 0 to 725
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   respondent      726 non-null    int64  
 1   task            726 non-null    int64  
 2   interval        726 non-null    object 
 3   time            726 non-null    float64
 4   company         726 non-null    float64
 5   purpose         725 non-null    object 
 6   purpose_eval    726 non-null    float64
 7   detail          726 non-null    float64
 8   description     725 non-null    object 
 9   desc_eval_comp  726 non-null    int64  
 10  desc_eval_elem  726 non-null    int64  
 11  desc_eval_char  726 non-null    int64  
 12  PQ1             726 non-null    int64  
 13  ATT1            726 non-null    int64  
 14  PQ2             726 non-null    int64  
 15  HQ1             726 non-null    int64  
 16  PQ3             726 non-null    int64  
 17  HQ2             726 non-null    int

In [4]:
results.head()

,respondent,task,interval,time,company,purpose,purpose_eval,detail,description,desc_eval_comp,...,ATT1,PQ2,HQ1,PQ3,HQ2,HQ3,ATT2,PQ4,HQ4,familiarity
0,3,1,2s,94.0,1.0,Clothing,1.0,0.0,There were pictures of people of different age...,1,...,7,7,6,6,5,6,7,7,7,False
1,3,2,2s,425.0,0.0,"Glasses, contact lenses, sunglasses",1.0,0.0,"Pictures of people smiling, some with sunglasses.",0,...,6,7,6,6,6,6,7,7,7,False
2,3,3,2s,73.0,1.0,Sport gear,1.0,0.0,There was a picture of a white leopard. There ...,1,...,7,7,6,5,6,6,7,7,6,False
3,3,4,2s,85.0,1.0,Information about the weather,1.0,0.0,There were lots of different weather forecasts...,1,...,7,7,6,7,5,6,7,7,6,False
4,3,5,2s,413.0,1.0,Pet supplies,1.0,1.0,There were different pictures of items for pets,0,...,5,6,4,5,4,5,6,6,4,False


**Add columns.**

In [5]:
results.insert(22, 'ATT', '')
results.insert(22, 'HQ', '')
results.insert(22, 'PQ', '')
results.insert(12, 'desc_eval_sum', '')
results.insert(9, 'desc_word_count_group', '')
results.insert(9, 'desc_word_count', '')
results.insert(9, 'desc_token_count_group', '')
results.insert(9, 'desc_token_count', '')
results.insert(9, 'desc_tokens', '')
results.insert(7, 'purpose_word_count', '')
results.insert(7, 'purpose_token_count', '')
results.insert(7, 'purpose_tokens', '')

In [6]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 0 to 725
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   respondent              726 non-null    int64  
 1   task                    726 non-null    int64  
 2   interval                726 non-null    object 
 3   time                    726 non-null    float64
 4   company                 726 non-null    float64
 5   purpose                 725 non-null    object 
 6   purpose_eval            726 non-null    float64
 7   purpose_tokens          726 non-null    object 
 8   purpose_token_count     726 non-null    object 
 9   purpose_word_count      726 non-null    object 
 10  detail                  726 non-null    float64
 11  description             725 non-null    object 
 12  desc_tokens             726 non-null    object 
 13  desc_token_count        726 non-null    object 
 14  desc_token_count_group  726 non-null    ob

**Calculate number of words and number of full meaning words.**

In [7]:
lemmatizer = WordNetLemmatizer()

for index, row in results.iterrows():
    if(type(row.description) == str):
        tokens = word_tokenize(row.description.replace('/', ' ').replace('-', ' '))
        tokens_no_punct = list(filter(lambda x: x.isalpha(), [word.lower() for word in tokens if word.isalpha()]))
        tokens_no_stop = [word for word in tokens_no_punct if not word in stopwords.words()]
        tokens_final = list(map(lambda x: lemmatizer.lemmatize(x), tokens_no_stop))
        results.loc[index, 'desc_word_count'] = len(tokens_no_punct)
        results.loc[index, 'desc_token_count'] = len(tokens_final)
        results.loc[index, 'desc_tokens'] = ' '.join(tokens_final)
    else:
        results.loc[index, 'desc_word_count'] = 0
        results.loc[index, 'desc_token_count'] = 0
    if(type(row.purpose) == str):
        tokens = word_tokenize(row.purpose.replace('/', ' ').replace('-', ' '))
        tokens_no_punct = list(filter(lambda x: x.isalpha(), [word.lower() for word in tokens if word.isalpha()]))
        tokens_no_stop = [word for word in tokens_no_punct if not word in stopwords.words()]
        tokens_final = list(map(lambda x: lemmatizer.lemmatize(x), tokens_no_stop))
        results.loc[index, 'purpose_word_count'] = len(tokens_no_punct)
        results.loc[index, 'purpose_token_count'] = len(tokens_final)
        results.loc[index, 'purpose_tokens'] = ' '.join(tokens_final)
    else:
        results.loc[index, 'purpose_word_count'] = 0
        results.loc[index, 'purpose_token_count'] = 0

In [8]:
results.columns

Index(['respondent', 'task', 'interval', 'time', 'company', 'purpose',
       'purpose_eval', 'purpose_tokens', 'purpose_token_count',
       'purpose_word_count', 'detail', 'description', 'desc_tokens',
       'desc_token_count', 'desc_token_count_group', 'desc_word_count',
       'desc_word_count_group', 'desc_eval_comp', 'desc_eval_elem',
       'desc_eval_char', 'desc_eval_sum', 'PQ1', 'ATT1', 'PQ2', 'HQ1', 'PQ3',
       'HQ2', 'HQ3', 'ATT2', 'PQ4', 'HQ4', 'PQ', 'HQ', 'ATT', 'familiarity'],
      dtype='object')

In [9]:
results.loc[results.desc_word_count <= 11, 'desc_word_count_group'] = 'low' 
results.loc[(results.desc_word_count > 11) & (results.desc_word_count < 24), 'desc_word_count_group'] = 'med' 
results.loc[results.desc_word_count >= 24, 'desc_word_count_group'] = 'high' 
results.groupby('desc_word_count_group').size()

desc_word_count_group
high    239
low     244
med     243
dtype: int64

In [10]:
results.loc[results.desc_token_count <= 5, 'desc_token_count_group'] = 'low' 
results.loc[(results.desc_token_count > 5) & (results.desc_token_count < 13), 'desc_token_count_group'] = 'med' 
results.loc[results.desc_token_count >= 13, 'desc_token_count_group'] = 'high' 
results.groupby('desc_token_count_group').size()

desc_token_count_group
high    207
low     216
med     303
dtype: int64

**Remove unecessary columns.**

In [11]:
results.purpose = results.purpose_eval

In [12]:
results = results.drop(columns={
    'description', 'purpose_eval'
})

In [13]:
results.head()

,respondent,task,interval,time,company,purpose,purpose_tokens,purpose_token_count,purpose_word_count,detail,...,PQ3,HQ2,HQ3,ATT2,PQ4,HQ4,PQ,HQ,ATT,familiarity
0,3,1,2s,94.0,1.0,1.0,clothing,1,1,0.0,...,6,5,6,7,7,7,,,,False
1,3,2,2s,425.0,0.0,1.0,glass contact lens sunglass,4,4,0.0,...,6,6,6,7,7,7,,,,False
2,3,3,2s,73.0,1.0,1.0,sport gear,2,2,0.0,...,5,6,6,7,7,6,,,,False
3,3,4,2s,85.0,1.0,1.0,information weather,2,4,0.0,...,7,5,6,7,7,6,,,,False
4,3,5,2s,413.0,1.0,1.0,supply,1,2,1.0,...,5,4,5,6,6,4,,,,False


**Means of Attrak-Diff results.**

In [14]:
results['PQ'] = (results.PQ1 + results.PQ2 + results.PQ3 + results.PQ4) / 4
results['HQ'] = (results.HQ1 + results.HQ2 + results.HQ3 + results.HQ4) / 4
results['ATT'] = (results.ATT1 + results.ATT2) / 2

**Sum of webpage elements and characteristics.**

In [15]:
results['desc_eval_sum'] = results['desc_eval_elem'] + results['desc_eval_char']

In [16]:
results.columns

Index(['respondent', 'task', 'interval', 'time', 'company', 'purpose',
       'purpose_tokens', 'purpose_token_count', 'purpose_word_count', 'detail',
       'desc_tokens', 'desc_token_count', 'desc_token_count_group',
       'desc_word_count', 'desc_word_count_group', 'desc_eval_comp',
       'desc_eval_elem', 'desc_eval_char', 'desc_eval_sum', 'PQ1', 'ATT1',
       'PQ2', 'HQ1', 'PQ3', 'HQ2', 'HQ3', 'ATT2', 'PQ4', 'HQ4', 'PQ', 'HQ',
       'ATT', 'familiarity'],
      dtype='object')

**Replace faulty results.**

In [17]:
for i in range(1, 7):
    results.loc[i + 677, 'time'] = np.floor(results[results.task == i].time.mean())

**Insert design complexities.**

In [18]:
complexities = {
    1: 0.22,
    2: 2,
    3: 2.97,
    4: 5.27,
    5: 7.5,
    6: 8
}

In [19]:
results.insert(2, 'complexity', 0)
for index, row  in results.iterrows():
    results.loc[index, 'complexity'] = complexities[row.task]

**Remove respondent 40**

In [20]:
results = results[(results.respondent != 40) | (results.interval != '2s')]

**Export.**

In [21]:
results.to_csv("../data/results.csv")

## Respondent cognitive tests calculations

**Load data.**

In [22]:
respondents = pd.read_csv("../data/respondents_to_calculate.csv", index_col=0)

**Remove respondents that did not finish and respondent 40**

In [23]:
respondents = respondents[(respondents.status == 'completed')]

In [24]:
respondents = respondents[(respondents.respondent != 40) | (respondents.interval != '2s')]

In [25]:
respondents.shape

(120, 31)

In [26]:
respondents.columns

Index(['respondent', 'identifier', 'status', 'included', 'started_at', 'ip',
       'device', 'os', 'browser', 'location', 'time_taken',
       'questions_answered', 'tasks_completed', 'tasks_skipped', 'vmt_levels',
       'vmt_clicks', 'pst_levels', 'pst_levels_correct', 'pst_clicks',
       'input_device', 'age', 'gender', 'education', 'income', 'web_browsing',
       'web_skill', 'att_check_1', 'similar_studies', 'note', 'att_check_2',
       'interval'],
      dtype='object')

In [27]:
respondents.insert(19, 'pst_press_speed', 0)
respondents.insert(18, 'pst_levels_correct_group', 0)
respondents.insert(17, 'pst_mistakes_group', '')
respondents.insert(17, 'pst_mistakes', 0)
respondents.insert(17, 'pst_levels_group', '')
respondents.insert(16, 'vmt_press_len_group', '')
respondents.insert(16, 'vmt_press_len', 0)
respondents.insert(16, 'vmt_click_speed_group', '')
respondents.insert(16, 'vmt_click_speed', 0)
respondents.insert(15, 'vmt_levels_group', '')

**Calculate press lengths and click speeds.**

In [28]:
for index, respondent in respondents.iterrows():
    if(type(respondent.vmt_clicks) == str):
        levels = json.loads(respondent.vmt_clicks)
        sum_time = 0
        sum_press_time = 0
        clicks = 0
        between_clicks = 0
        for level in levels:
            last_time = 0
            for click in level:
                if(click['type']=='mouseDown'):
                    sum_time += (click['time'] - last_time) if last_time != 0 else 0
                    clicks += 1
                    between_clicks += 1 if last_time != 0 else 0
                    last_time = click['time']
                if(click['type']=='mouseUp'):
                    sum_press_time += (click['time'] - last_time)
        
        respondents.loc[index, 'vmt_click_speed'] = (between_clicks / sum_time ) * 1000
        respondents.loc[index, 'vmt_press_len'] = sum_press_time / clicks

In [29]:
for index, respondent in respondents.iterrows():
    if(type(respondent.pst_clicks) == str):
        presses = json.loads(respondent.pst_clicks)
        sum_time = presses[len(presses) - 1] - presses[0]
        respondents.loc[index, 'pst_press_speed'] = (len(presses) / sum_time ) * 1000

**Assign groups according to cognitive tasks**

In [30]:
respondents['pst_mistakes'] = respondents['pst_levels'] - respondents['pst_levels_correct']
respondents['pst_mistakes'] = respondents['pst_mistakes']

In [31]:
respondents.loc[respondents.vmt_levels <= 5, 'vmt_levels_group'] = 'low' 
respondents.loc[(respondents.vmt_levels > 5) & (respondents.vmt_levels < 8), 'vmt_levels_group'] = 'med' 
respondents.loc[respondents.vmt_levels >= 8, 'vmt_levels_group'] = 'high' 
respondents.groupby('vmt_levels_group').size()

vmt_levels_group
high    30
low     29
med     61
dtype: int64

In [32]:
respondents.loc[respondents.pst_levels <= 30, 'pst_levels_group'] = 'low' 
respondents.loc[(respondents.pst_levels > 30) & (respondents.pst_levels < 38), 'pst_levels_group'] = 'med' 
respondents.loc[respondents.pst_levels >= 38, 'pst_levels_group'] = 'high' 
respondents.groupby('pst_levels_group').size()

pst_levels_group
high    40
low     36
med     44
dtype: int64

In [33]:
respondents.loc[respondents.pst_levels_correct <= 27, 'pst_levels_correct_group'] = 'low' 
respondents.loc[(respondents.pst_levels_correct > 27) & (respondents.pst_levels_correct < 36), 'pst_levels_correct_group'] = 'med' 
respondents.loc[respondents.pst_levels_correct >= 36, 'pst_levels_correct_group'] = 'high' 
respondents.groupby('pst_levels_correct_group').size()

pst_levels_correct_group
high    35
low     36
med     49
dtype: int64

In [34]:
respondents.loc[respondents.pst_mistakes <= 0, 'pst_mistakes_group'] = 'low' 
respondents.loc[(respondents.pst_mistakes > 0) & (respondents.pst_mistakes < 3), 'pst_mistakes_group'] = 'med' 
respondents.loc[respondents.pst_mistakes >= 3, 'pst_mistakes_group'] = 'high' 
respondents.groupby('pst_mistakes_group').size()

pst_mistakes_group
high    36
low     27
med     57
dtype: int64

In [35]:
respondents.loc[respondents.vmt_click_speed <= 1.34, 'vmt_click_speed_group'] = 'low' 
respondents.loc[(respondents.vmt_click_speed > 1.34) & (respondents.vmt_click_speed < 1.7), 'vmt_click_speed_group'] = 'med' 
respondents.loc[respondents.vmt_click_speed >= 1.7, 'vmt_click_speed_group'] = 'high' 
respondents.groupby('vmt_click_speed_group').size()

vmt_click_speed_group
high    40
low     40
med     40
dtype: int64

In [36]:
respondents.loc[respondents.vmt_press_len <= 82.3, 'vmt_press_len_group'] = 'low' 
respondents.loc[(respondents.vmt_press_len > 82.3) & (respondents.vmt_press_len < 112), 'vmt_press_len_group'] = 'med' 
respondents.loc[respondents.vmt_press_len >= 112, 'vmt_press_len_group'] = 'high' 
respondents.groupby('vmt_press_len_group').size()

vmt_press_len_group
high    39
low     40
med     41
dtype: int64

**Remove uneccessary columns.**

In [37]:
respondents = respondents.drop(columns={
    'vmt_clicks', 'pst_clicks'
})

In [38]:
respondents.columns

Index(['respondent', 'identifier', 'status', 'included', 'started_at', 'ip',
       'device', 'os', 'browser', 'location', 'time_taken',
       'questions_answered', 'tasks_completed', 'tasks_skipped', 'vmt_levels',
       'vmt_levels_group', 'vmt_click_speed', 'vmt_click_speed_group',
       'vmt_press_len', 'vmt_press_len_group', 'pst_levels',
       'pst_levels_group', 'pst_mistakes', 'pst_mistakes_group',
       'pst_levels_correct', 'pst_levels_correct_group', 'pst_press_speed',
       'input_device', 'age', 'gender', 'education', 'income', 'web_browsing',
       'web_skill', 'att_check_1', 'similar_studies', 'note', 'att_check_2',
       'interval'],
      dtype='object')

In [39]:
respondents.head()

,respondent,identifier,status,included,started_at,ip,device,os,browser,location,...,gender,education,income,web_browsing,web_skill,att_check_1,similar_studies,note,att_check_2,interval
0,3,NaN,completed,True,2022-09-30 14:21:47,NaN,desktop,Windows,Edge,GB,...,Woman,Undergraduate degree,"£10,000 - £19,999",Few times a day,3.0,Salad,"Yes, I've done studies where I've completed a ...",NaN,30.0,2s
1,4,NaN,completed,True,2022-09-30 14:22:50,NaN,desktop,OS X,Safari,GB,...,Man,Graduate degree,"£30,000 - £39,999",Most of the day,5.0,Salad,NaN,NaN,30.0,2s
2,5,NaN,completed,True,2022-09-30 14:26:56,NaN,desktop,Windows,Chrome,GB,...,Man,Technical/community college,"£20,000 - £29,999",Few times a day,5.0,Salad,yes but not the same,no thanks,30.0,2s
4,7,NaN,completed,True,2022-10-06 12:51:06,NaN,desktop,OS X,Chrome,GB,...,Woman,Technical/community college,"£10,000 - £19,999",Most of the day,4.0,Salad,I don't think so.,NaN,30.0,2s
5,8,NaN,completed,True,2022-10-06 12:57:21,NaN,desktop,Windows,Chrome,GB,...,Woman,Undergraduate degree,"£0 - £10,000",Most of the day,3.0,Salad,No,No,30.0,2s


**Export.**

In [40]:
respondents.to_csv("../data/respondents.csv")